In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
import pickle

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms

In [ ]:
test = torch.load('../input/encodings/instruction_embeddings.pt')

In [ ]:
test = pad_sequence(test, batch_first=True)

In [ ]:
instructions = torch.load('../input/10k-encodings/instruction_embeddings_10k.pt')

In [ ]:
instructions = instructions[0:5000]

In [ ]:
len(instructions)

In [ ]:
instructions = pad_sequence(instructions, batch_first=True)

In [ ]:
with open("../input/10k-encodings/lengths_10k.txt", "rb") as fp:
    lengths = pickle.load(fp)

In [ ]:
lengths = lengths[0:5000]

In [ ]:
with open("../input/encodings/lengths.txt", "rb") as fp:
    lengths_test = pickle.load(fp)

In [ ]:
lengths_test = torch.FloatTensor(lengths_test)

In [ ]:
len(instructions[0])

In [ ]:
import numpy as np

In [ ]:
image_encoding = np.load('../input/encodings/train_image_encoding_2048_1000.npy')

In [ ]:
np.append([[1,2],[2,3]],[[4,5],[5,6]],axis=0)

In [ ]:
import os
directory = '../input/train-image-encoding-2048-2000-5000'

for i in range(2000,6000,1000):
    print(i)
    
    temp = np.load('../input/train-image-encoding-2048-2000-5000/train_image_encoding_2048_'+str(i)+'.npy')
    image_encoding = np.append(image_encoding,temp,axis=0)


In [ ]:

#directory = '../input/train-image-encoding-2048-6000-10000'

#for i in range(6000,11000,1000):
#    print(i)
    
#    temp = np.load('../input/train-image-encoding-2048-6000-10000/train_image_encoding_2048_'+str(i)+'.npy')
#    image_encoding = np.append(image_encoding,temp,axis=0)


In [ ]:
image_encoding.shape[0]

In [ ]:
import pandas as pd

In [ ]:
image_test = torch.from_numpy(image_encoding[:1000])

In [ ]:
data2 = pd.DataFrame(instructions)

In [ ]:
data1 = pd.DataFrame(lengths)

In [ ]:
data3 = pd.DataFrame(image_encoding)

In [ ]:
frames = [data2,data1,data3]
data = pd.concat(frames,axis=1)

In [ ]:
data.head

In [ ]:
class Data(Dataset):
    
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = torch.tensor(self.data.iloc[index, 2:].values.astype(np.float32))
        instructions = self.data.iloc[index, 0]
        lengths = self.data.iloc[index,1]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return instructions, lengths, image

In [ ]:
train_dataset = Data(data, transform=None)

In [ ]:
instruction, length, image = train_dataset.__getitem__(0)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
train_iter = iter(train_loader)
print(type(train_iter))

In [ ]:
instructions, lengths, images = train_iter.next()

print('images shape on batch size = {}'.format(images.size()))
print('instructions shape on batch size = {}'.format(instructions.size()))
print('lengths shape on batch size = {}'.format(lengths.size()))

In [ ]:
len(lengths)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
  
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
import torch
import numpy as np


In [ ]:
def l2norm(X):
    """L2-normalize columns of X
    """
    norm = torch.pow(X, 2).sum(dim=1, keepdim=True).sqrt()
    X = torch.div(X, norm)
    return X

In [ ]:
def accuracy(recipe,images):
    accuracy = 0
    r = torch.matmul(recipe, torch.transpose(images,0,1))
    top_values, top_indices = torch.topk(r,10)
    for i in range(len(recipe)):
        for boolean in top_indices[i] == i:
            if boolean:
                accuracy+=1
                break
    accuracy = (accuracy/len(recipe))*100
    return accuracy
                 

In [ ]:
class TextEncoder(nn.Module):

    def __init__(self, embed_dim, hidden_size, output_size, dropout, num_layers, use_abs=False):
        super(TextEncoder, self).__init__()
        self.use_abs = use_abs
        self.hidden_size = hidden_size

        
        self.rnn = nn.GRU(embed_dim, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

        self.init_weights()

    def init_weights(self):
        # Linear
        r = np.sqrt(6.) / np.sqrt(self.fc.in_features +
                                  self.fc.out_features)
        self.fc.weight.data.uniform_(-r, r)
        self.fc.bias.data.fill_(0)

        

    def forward(self, x, lengths):
        """Handles variable size captions
        """
       
        lengths = torch.FloatTensor(lengths)
        # pytorch 1.1
        packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        
        out, _ = self.rnn(packed)
        
        padded, _ = pad_packed_sequence(out, batch_first=True)

        # Use final LSTM Output
        length_tensor = lengths.reshape(-1, 1, 1)  # shape: bs,1,1
        # Get last index (before padding) of each sentence
        
        length_tensor = length_tensor.expand(x.size(0), 1, self.hidden_size) - 1  # bs, 1, hidden_size
        length_tensor = torch.tensor(length_tensor,dtype=torch.int64)
        
        out = torch.gather(padded, 1, length_tensor).squeeze(1)  # bs * hidden_size

        # normalization in the joint embedding space
        output = l2norm(out)
        output = self.fc(self.dropout(output))

        # take absolute value, used by order embeddings
        if self.use_abs:
            output = torch.abs(out)

        return output

In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.optim import Adam, SGD

In [ ]:
embed_dim = 4096
hidden_size = 5000
output_size = 2048
dropout = 0
num_layers = 1
max_epochs = 15

In [ ]:
lr = 0.1

In [ ]:
y = torch.ones([128], dtype=torch.int32)
# y is a tensor of 1 and -1, indicating whether it's a positive pair or negative pair

In [ ]:
model = TextEncoder(embed_dim = 4096, hidden_size = 5000, output_size =2048, dropout = 0, num_layers = 1)
model.load_state_dict(torch.load('../input/trained-model/model.pt'))
loss_function = nn.CosineEmbeddingLoss()
params = list(model.parameters())
optimizer = Adam(params, lr=lr, weight_decay=5e-4)


In [ ]:
model.train()

In [ ]:
out = open('./model.pt', 'w')   

In [ ]:
temp = 1.0
for epoch in range(max_epochs):
    print("Epoch: ",epoch+1,"/",max_epochs,end= " ")
    train_iter = iter(train_loader)
    for instructions, lengths, images in train_iter:
        model.zero_grad()
        output = model(instructions,lengths.float())
        loss = loss_function(output,images, torch.ones([len(lengths)], dtype=torch.int32))
        loss.backward()
        optimizer.step()
        print("-",end = "")
    with torch.no_grad():
        recipe = model(test, lengths_test)
    max_accuracy = accuracy(recipe,image_test.type(torch.FloatTensor))
    print("> Loss: ",loss," Accuracy: ",max_accuracy)
    if (max_accuracy>temp):
        torch.save(model.state_dict(), './model.pt')
        temp = max_accuracy
    
     

In [ ]:
with torch.no_grad():
        recipe = model(test, lengths_test)
accuracy(recipe,image_test.type(torch.FloatTensor))